# ReqRespBunch


In [ ]:
#| default_exp ReqRespBunch

In [ ]:
#| export 
from FastRequest.RestReqFactory import *
from fastcore.basics import patch
import requests
import json

In [ ]:
#| export
from typing import List
class ReqRespBunch:
    """
    given reqs_lis of type `RestReqFactory`, itcreates response and kwargs for each element of list  
    provides factorey method for getter as well as setter forthe respnse and kwargs set for each request
    """
    def __init__(self, 
                 reqs_lis:List['FastRequest.RestReqFactory.RestReqFactory']=None # list of RestReqFactory 
                ):
        """
        Initializes a ReqRespBunch object.

        Args:
            reqs_lis (list): List of requests to be stored in the object.

        Returns:
            None
        """
        self._reqs = reqs_lis
        self._resps = [None] * len(reqs_lis)
        self._kwargs = [{} for _ in range(len(reqs_lis))]

    @property
    def reqs(self):
        """
        Returns the list of requests stored in the object.

        Returns:
            list: List of requests.
        """
        return self._reqs

    @property
    def resps(self):
        """
        Returns the list of responses stored in the object.

        Returns:
            list: List of responses.
        """
        return self._resps

    @property
    def kwargs(self):
        """
        Returns the list of kwargs (arguments) associated with each request.

        Returns:
            list: List of dictionaries containing kwargs for each request.
        """
        return self._kwargs

    def set_resp(self, idx, val):
        """
        Sets the response at the specified index in the response list.

        Args:
            idx (int): Index at which to set the response.
            val: Value to set as the response.

        Returns:
            None
        """
        self._resps[idx] = val

    def set_kwargs(self, idx, val):
        """
        Sets the kwargs (arguments) for the request at the specified index.

        Args:
            idx (int): Index of the request.
            val (tuple): Tuple containing key-value pair to set as kwargs.

        Returns:
            None
        """
        self._kwargs[idx][val[0]] = val[1]

    def __len__(self):
        return len(self.reqs)
        
    def __iter__(self):
        self._idx = 0
        return self

    def __next__(self):
        if self._idx <  len(self.kwargs):
            result = (self._reqs[self._idx], self._resps[self._idx])
            self._idx += 1
            return result
        else:
            raise StopIteration

    @property
    def print_req_resp(self):
        " returns the request and response in nice formatted way"
        data = []
        for req, res in self:
            req_str = req()
            res_str = res
    
    
            data.append({
                'request': req_str,
                'response': res_str})
    
    
        return data

In [ ]:
# Create the RestReqFactory instance
req1 = RestReqFactory(
    method="GET",
    url_provider= lambda: f"{get_env('url1')}/get",
    params_provider= lambda : {
        "foo1": f"{get_env('foo1')}",
        "foo2": f"{get_env('foo2')}",
    }
)
req2 = RestReqFactory(
    method="GET",
    url_provider= lambda : f"{get_env('url2')}/api/users/2",
)
reqs = ReqRespBunch([req1, req2])
len(reqs)

2

In [ ]:
req1()

{
    "method": "GET",
    "url": "/get",
    "headers": "None",
    "params": {
        "foo1": "",
        "foo2": ""
    },
    "body": "None",
    "kwargs": "None"
}

In [ ]:
set_env('foo1', 'foo1')
set_env('foo2', 'foo2')
set_env('url1', 'https://postman-echo.com')
set_env('url2', 'https://reqres.in')

In [ ]:
#| export
@patch
def __repr__(self:requests.models.Response):
    " monkey patch the __repr__ of the repsone object"
    response_info = {
            'url': self.url,
            'status': self.status_code,
            'headers': dict(self.headers),
            'cookies': self.cookies.get_dict(),
            'elapsed': str(self.elapsed),
            'text': self.text[:1000]  # Limit to first 1000 characters
        }

    # If the response contains JSON, include it in the output
    try:
        response_info['json'] = self.json()
    except json.JSONDecodeError:
        response_info['json'] = 'No JSON data'

    return json.dumps(response_info, indent=4)

@patch
def __str__(self:requests.models.Response):
    " monkey patch the __str__ of the repsone object"
    
    return self.__repr__()

In [ ]:
resp = req1()()
type(resp)

requests.models.Response

In [ ]:
reqs.set_resp(0, req1()())
reqs.set_resp(1, req2()())

In [ ]:
#| hide
assert reqs.resps[0] != str(req1()()) # as various object will change like Date and ETag 

### Printing requset and response  in a pair in json  format

In [ ]:
reqs.print_req_resp

[{'request': {
      "method": "GET",
      "url": "https://postman-echo.com/get",
      "headers": "None",
      "params": {
          "foo1": "foo1",
          "foo2": "foo2"
      },
      "body": "None",
      "kwargs": "None"
  },
  'response': {
      "url": "https://postman-echo.com/get?foo1=foo1&foo2=foo2",
      "status": 200,
      "headers": {
          "Date": "Mon, 10 Jun 2024 14:31:51 GMT",
          "Content-Type": "application/json; charset=utf-8",
          "Content-Length": "512",
          "Connection": "close",
          "Server": "nginx/1.25.3",
          "ETag": "W/\"200-3Jb3WQoBKDrr9SPJsIdfXrK4rlY\"",
          "set-cookie": "sails.sid=s%3A5xSPUWYL9dBQ2vDPzrHJgPTl4ULeKY6x.vDQ%2F42S8EmWnOswcfTV8dh0lr0Be6Z9mxokjmKw1uF4; Path=/; HttpOnly"
      },
      "cookies": {
          "sails.sid": "s%3A5xSPUWYL9dBQ2vDPzrHJgPTl4ULeKY6x.vDQ%2F42S8EmWnOswcfTV8dh0lr0Be6Z9mxokjmKw1uF4"
      },
      "elapsed": "0:00:00.930238",
      "text": "{\n  \"args\": {\n    \"foo1\": \"fo

In [ ]:
#| hide
import nbdev

In [ ]:
#| hide
nbdev.export.nb_export('ReqRespBunch.ipynb','../FastRequest/')